# Code to Join together EIA, EPA Data and Generate Emissions Rates

In [245]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

os.chdir('C:/Users/lbeatty/Documents/Lauren_MIP_Contribution/')

In [246]:
####################
# Read in Data #####
####################

crosswalk = pd.read_csv("Data/epa_eia_crosswalk.csv")
emissions = pd.read_csv("Data/CAMD/facilities_emissions.csv")
#only want year 2020
emissions = emissions.query('year==2020')

# PM25
pm25 = pd.read_excel("Data/eGRID2020 DRAFT PM Emissions.xlsx", sheet_name="2020 PM Unit-level Data", skiprows=1)

# EIA 923 For Annual Generation
eia923 = pd.read_excel("Data/eia923/EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx", sheet_name='Page 4 Generator Data', skiprows=5)

# EIA 860 for Generator Info
eia860 = pd.read_excel("Data/eia860/3_1_Generator_Y2020.xlsx", skiprows=1)

In [247]:
################
## Format Data #
################
pm25['UNITID'] = pm25['UNITID'].astype(str)
pm25 = pm25.groupby(['ORISPL', 'UNITID']).agg({'PM25AN': 'sum'}).reset_index()
pm25.columns = ['facilityId', 'unitId', 'pm25']
pm25['unitId']=pm25['unitId'].astype(str)
pm25['facilityId']=pm25['facilityId'].astype(int)


emissions['unitId']=emissions['unitId'].astype(str)
emissions['facilityId']=emissions['facilityId'].astype(int)

crosswalk = crosswalk[['CAMD_PLANT_ID', 'CAMD_UNIT_ID', 'CAMD_GENERATOR_ID', 'EIA_PLANT_ID', 'EIA_GENERATOR_ID', 'EIA_UNIT_TYPE']]
crosswalk['EIA_GENERATOR_ID'] = crosswalk['EIA_GENERATOR_ID'].astype(str)

eia923 = eia923[['Plant Id', 'Generator Id', 'Plant Name', 'Plant State', 'Net Generation\nYear To Date']]
eia923.columns = ['EIA_PLANT_ID', 'EIA_GENERATOR_ID', 'EIA_PLANT_NAME', 'EIA_STATE', 'NET_GEN']
eia923['EIA_GENERATOR_ID']=eia923['EIA_GENERATOR_ID'].astype(str)

eia860 = eia860[['Plant Code', 'Generator ID', 'Nameplate Capacity (MW)', 'Planned Retirement Year', 'Planned Retirement Month', 'Synchronized to Transmission Grid']]
eia860.columns = ['EIA_PLANT_ID', 'EIA_GENERATOR_ID', 'Capacity', 'RetirementYear', 'RetirementMonth', 'SynchronizedToGrid']
eia860['EIA_GENERATOR_ID']=eia860['EIA_GENERATOR_ID'].astype(str)

print(eia860.dtypes)
print(eia923.dtypes)
print(pm25.dtypes)
print(emissions.dtypes)
print(crosswalk.dtypes)

EIA_PLANT_ID          float64
EIA_GENERATOR_ID       object
Capacity              float64
RetirementYear         object
RetirementMonth        object
SynchronizedToGrid     object
dtype: object
EIA_PLANT_ID          int64
EIA_GENERATOR_ID     object
EIA_PLANT_NAME       object
EIA_STATE            object
NET_GEN             float64
dtype: object
facilityId      int32
unitId         object
pm25          float64
dtype: object
stateCode             object
facilityName          object
facilityId             int32
unitId                object
associatedStacks      object
year                   int64
countOpTime            int64
sumOpTime            float64
grossLoad            float64
steamLoad            float64
so2Mass              float64
so2Rate              float64
co2Mass              float64
co2Rate              float64
noxMass              float64
noxRate              float64
heatInput            float64
primaryFuelInfo       object
secondaryFuelInfo     object
unitType             

### Merge Data Together

Merge crosswalk into EIA and EPA data, then merge together.

Logic here is confusing...

Emissions are reported at the 'unit' level which can aggregate multiple EIA boilers (and visa-versa).  Therefore, the strategy will be to join the crosswalk into EIA data, sum generation and capacity within CAMD 'units,' then calculate CAMD 'unit' level emissions rate.  

Then merge emissions rates BACK into EIA data and output to a csv file.

In [248]:
# Join EIA Together
eia_joined = pd.merge(eia923, eia860, on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], how='left')

# Merge crosswalk into eia
eia_joined = pd.merge(eia_joined, crosswalk, on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], how='left')

#Collapse EIA to CAMD unit
eia_collapsed = eia_joined.groupby(['CAMD_PLANT_ID', 'CAMD_UNIT_ID']).agg({'NET_GEN': 'sum', 'Capacity': 'sum'}).reset_index()

# Join CAMD emissions with PM25
emissions = pd.merge(emissions, pm25, on=['facilityId', 'unitId'], how='left')

# Join with CAMD
eia_collapsed = eia_collapsed.rename(columns={'CAMD_PLANT_ID' : 'facilityId', 'CAMD_UNIT_ID' : 'unitId'})
camd_eia_data = pd.merge(emissions, eia_collapsed, on=['facilityId', 'unitId'], how='left')


In [249]:
### Calculate tons
# Calculate values based on existing columns
camd_eia_data['nox_lbs'] = camd_eia_data['noxMass'] * 2000
camd_eia_data['nox_rate'] = camd_eia_data['nox_lbs'] / camd_eia_data['NET_GEN']
camd_eia_data['so2_lbs'] = camd_eia_data['so2Mass'] * 2000
camd_eia_data['so2_rate'] = camd_eia_data['so2_lbs'] / camd_eia_data['NET_GEN']
camd_eia_data['pm25_lbs'] = camd_eia_data['pm25'] * 2000
camd_eia_data['pm25_rate'] = camd_eia_data['pm25_lbs'] / camd_eia_data['NET_GEN']

In [250]:
## Now merge crosswalk back into camd_eia_data
crosswalk = crosswalk.rename(columns = {'CAMD_PLANT_ID': 'facilityId', 'CAMD_UNIT_ID': 'unitId'})
camd_eia_data = pd.merge(camd_eia_data, crosswalk, on=['facilityId', 'unitId'], how='left')

## Calculate Generation-weighted emissions rates

Trick here is that there's lots of missings.  First I'm going to throw out all rows with relevant missings, compute emissions-rates by EIA generator.  Then I'm going to find rows where some of the relevant data is there, and again compute emissions-rates by EIA generator.  Then I'm going to calculate emissions rates by EIA plan.  Finally I'll coalesce the data so that the most precise numbers are taken first.

In [251]:
# Create Generation-weighted eia-generator level emissions rates
camd_eia_data['noxrate_generation'] = camd_eia_data['nox_rate']*camd_eia_data['NET_GEN']
camd_eia_data['so2rate_generation'] = camd_eia_data['so2_rate']*camd_eia_data['NET_GEN']
camd_eia_data['pm25rate_generation'] = camd_eia_data['pm25_rate']*camd_eia_data['NET_GEN']


### start with all rows that don't have nan in NET_GEN
collapsed_eia_generator = camd_eia_data.query('NET_GEN.notnull()')\
    .groupby(['EIA_GENERATOR_ID', 'EIA_PLANT_ID'], as_index=False)\
    .agg(lambda x: np.nan if x.isnull().any() else x.sum())

collapsed_eia_generator['so2_rate']=collapsed_eia_generator['so2rate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator['pm25_rate']=collapsed_eia_generator['pm25rate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator['nox_rate']=collapsed_eia_generator['noxrate_generation']/collapsed_eia_generator['NET_GEN']

collapsed_eia_generator = collapsed_eia_generator[['EIA_GENERATOR_ID', 'EIA_PLANT_ID', 'NET_GEN', 'nox_rate', 'pm25_rate', 'so2_rate']]

## make a dataframe of all eia plants/generators and start left mergining in rates, sequentially filling in missings from best calcs to heaviest imputation
eia_emissions_rates = eia860[['EIA_PLANT_ID', 'EIA_GENERATOR_ID']]
eia_emissions_rates = pd.merge(eia_emissions_rates, collapsed_eia_generator, on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], how='left')


In [252]:
#First level of imputation still collapses within the generator but filters out missings first

#filter to has pm25rate
collapsed_eia_generator = camd_eia_data.query('NET_GEN.notnull()&pm25_rate.notnull()')\
    .groupby(['EIA_GENERATOR_ID', 'EIA_PLANT_ID'], as_index=False)\
    .agg(lambda x: np.nan if x.isnull().any() else x.sum())
collapsed_eia_generator['pm25_rate']=collapsed_eia_generator['pm25rate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator = collapsed_eia_generator[['EIA_GENERATOR_ID', 'EIA_PLANT_ID',  'pm25_rate']]
collapsed_eia_generator.columns = ['EIA_GENERATOR_ID', 'EIA_PLANT_ID', 'pm25_rate_impute1']

eia_emissions_rates = pd.merge(eia_emissions_rates, collapsed_eia_generator, on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], how='left')

#filter to has noxrate
collapsed_eia_generator = camd_eia_data.query('NET_GEN.notnull()&nox_rate.notnull()')\
    .groupby(['EIA_GENERATOR_ID', 'EIA_PLANT_ID'], as_index=False)\
    .agg(lambda x: np.nan if x.isnull().any() else x.sum())
collapsed_eia_generator['nox_rate']=collapsed_eia_generator['noxrate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator = collapsed_eia_generator[['EIA_GENERATOR_ID', 'EIA_PLANT_ID',  'nox_rate']]
collapsed_eia_generator.columns = ['EIA_GENERATOR_ID', 'EIA_PLANT_ID', 'nox_rate_impute1']

eia_emissions_rates = pd.merge(eia_emissions_rates, collapsed_eia_generator, on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], how='left')

#filter to has so2
collapsed_eia_generator = camd_eia_data.query('NET_GEN.notnull()&so2_rate.notnull()')\
    .groupby(['EIA_GENERATOR_ID', 'EIA_PLANT_ID'], as_index=False)\
    .agg(lambda x: np.nan if x.isnull().any() else x.sum())
collapsed_eia_generator['pm25_rate']=collapsed_eia_generator['so2rate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator = collapsed_eia_generator[['EIA_GENERATOR_ID', 'EIA_PLANT_ID',  'so2_rate']]
collapsed_eia_generator.columns = ['EIA_GENERATOR_ID', 'EIA_PLANT_ID', 'so2_rate_impute1']

eia_emissions_rates = pd.merge(eia_emissions_rates, collapsed_eia_generator, on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], how='left')

#didn't help all that much

In [253]:
#Next level is to collapse by EIA plant

#filter to has pm25rate
collapsed_eia_generator = camd_eia_data.query('NET_GEN.notnull()&pm25_rate.notnull()')\
    .groupby(['EIA_PLANT_ID'], as_index=False)\
    .agg(lambda x: np.nan if x.isnull().any() else x.sum())
collapsed_eia_generator['pm25_rate']=collapsed_eia_generator['pm25rate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator = collapsed_eia_generator[['EIA_PLANT_ID',  'pm25_rate']]
collapsed_eia_generator.columns = [ 'EIA_PLANT_ID', 'pm25_rate_impute2']

eia_emissions_rates = pd.merge(eia_emissions_rates, collapsed_eia_generator, on=['EIA_PLANT_ID'], how='left')

#filter to has noxrate
collapsed_eia_generator = camd_eia_data.query('NET_GEN.notnull()&nox_rate.notnull()')\
    .groupby(['EIA_PLANT_ID'], as_index=False)\
    .agg(lambda x: np.nan if x.isnull().any() else x.sum())
collapsed_eia_generator['nox_rate']=collapsed_eia_generator['noxrate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator = collapsed_eia_generator[['EIA_PLANT_ID',  'nox_rate']]
collapsed_eia_generator.columns = ['EIA_PLANT_ID', 'nox_rate_impute2']

eia_emissions_rates = pd.merge(eia_emissions_rates, collapsed_eia_generator, on=['EIA_PLANT_ID'], how='left')

#filter to has so2
collapsed_eia_generator = camd_eia_data.query('NET_GEN.notnull()&so2_rate.notnull()')\
    .groupby(['EIA_PLANT_ID'], as_index=False)\
    .agg(lambda x: np.nan if x.isnull().any() else x.sum())
collapsed_eia_generator['pm25_rate']=collapsed_eia_generator['so2rate_generation']/collapsed_eia_generator['NET_GEN']
collapsed_eia_generator = collapsed_eia_generator[[ 'EIA_PLANT_ID',  'so2_rate']]
collapsed_eia_generator.columns = ['EIA_PLANT_ID', 'so2_rate_impute2']

eia_emissions_rates = pd.merge(eia_emissions_rates, collapsed_eia_generator, on=['EIA_PLANT_ID'], how='left')


## I'm calling that a day for now

In [254]:
eia_emissions_rates['pm25_rate_imputed']=eia_emissions_rates['pm25_rate'].combine_first(eia_emissions_rates['pm25_rate_impute1']).combine_first(eia_emissions_rates['pm25_rate_impute2'])
eia_emissions_rates['so2_rate_imputed']=eia_emissions_rates['so2_rate'].combine_first(eia_emissions_rates['so2_rate_impute1']).combine_first(eia_emissions_rates['so2_rate_impute2'])
eia_emissions_rates['nox_rate_imputed']=eia_emissions_rates['nox_rate'].combine_first(eia_emissions_rates['nox_rate_impute1']).combine_first(eia_emissions_rates['nox_rate_impute2'])

#theres a couple emissions rates less than zero
#just make them zero for now

eia_emissions_rates['pm25_rate_imputed']=eia_emissions_rates['pm25_rate_imputed'].clip(lower=0)
eia_emissions_rates['so2_rate_imputed']=eia_emissions_rates['so2_rate_imputed'].clip(lower=0)
eia_emissions_rates['nox_rate_imputed'] = eia_emissions_rates['nox_rate_imputed'].clip(lower=0)

eia_emissions_rates = eia_emissions_rates[['EIA_PLANT_ID', 'EIA_GENERATOR_ID', 'NET_GEN', 'pm25_rate_imputed', 'so2_rate_imputed', 'nox_rate_imputed']]


In [257]:
eia_emissions_rates.to_csv('generator_emissions_rates.csv', index=False)